In [ ]:
import os
from os import path

# Third-party
from astropy.io import fits
from astropy.stats import median_absolute_deviation
from astropy.table import Table, QTable, join
from astropy.time import Time
import astropy.units as u
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import numpy as np
%matplotlib inline
import h5py
import pandas as pd
from sqlalchemy import func
import tqdm

from thejoker import JokerSamples

from twoface.config import TWOFACE_CACHE_PATH
from twoface.samples_analysis import unimodal_P, MAP_sample
from twoface.db import (db_connect, AllStar, AllVisit, AllVisitToAllStar, NessRG,
                        StarResult, Status, JokerRun, initialize_db)
from twoface.plot import plot_two_panel, plot_phase_fold, _RV_LBL
from twoface.mass import get_m2_min, mf, period_at_surface

In [ ]:
plot_path = '../../paper/1-catalog/figures/'
table_path = '../../paper/1-catalog/tables/'

Session, _ = db_connect(path.join(TWOFACE_CACHE_PATH, 'apogee.sqlite'))
session = Session()

samples_file = path.join(TWOFACE_CACHE_PATH, 'apogee-jitter.hdf5')
mcmc_samples_file = path.join(TWOFACE_CACHE_PATH, 'apogee-jitter-mcmc.hdf5')

In [ ]:
run = session.query(JokerRun).limit(1).one()
joker_pars = run.get_joker_params()

In [ ]:
full_catalog = QTable.read(path.join(table_path, 'high-K-unimodal.fits'), 
                           astropy_native=True)

In [ ]:
# cat = full_catalog
cat = full_catalog[(full_catalog['P_err']/full_catalog['P']) < 0.2]

fig, ax = plt.subplots(1, 1, figsize=(6, 6))

ax.scatter(cat['P'], cat['LOGG'], s=15)
ax.errorbar(cat['P'].value, cat['LOGG'],
            xerr=cat['P_err'].value, 
            yerr=cat['LOGG_ERR'],
            marker='', linestyle='none', zorder=-10, 
            alpha=0.7, color='#aaaaaa')

loggs = np.linspace(0, 4, 128)
_style = dict(marker='', zorder=-100)
P_surface = period_at_surface(1.35*u.Msun, loggs, 0., M2=0.4*u.Msun)
ax.plot(P_surface, loggs, ls='-', color='tab:orange', **_style)
P_surface = period_at_surface(1.35*u.Msun, loggs, 0.5, M2=0.4*u.Msun)
ax.plot(P_surface, loggs, ls='--', color='tab:orange', **_style)

ax.set_xlim(0.8, 2500)
ax.set_ylim(4, 0)
ax.set_xscale('log')

ax.set_xlabel('period, $P$ [day]')
ax.set_ylabel(r'$\log g$')

In [ ]:
# cat = full_catalog
cat = full_catalog[full_catalog['emcee_converged'] & (full_catalog['LOGG'] > -9999)]
# cat = full_catalog[full_catalog['emcee_converged'] & 
#                    np.logical_not(np.isin(full_catalog['APOGEE_ID'], suspicious_ids))]

fig, ax = plt.subplots(1, 1, figsize=(6, 6))

P_surface = period_at_surface(1.35*u.Msun, cat['LOGG'], cat['e'], 0.4*u.Msun)

ax.scatter(cat['P']/P_surface, cat['e'])
# ax.errorbar((full_catalog['P']/P_surface).value, full_catalog['LOGG'],
#             xerr=(full_catalog['P_err']/P_surface).value, 
#             yerr=full_catalog['LOGG_ERR'],
#             marker='', linestyle='none', zorder=-10)

ax.axvline(1., color='#aaaaaa', zorder=-100)
# ax.axvline(10., color='#aaaaaa', zorder=-100)

ax.set_xlabel(r'$P / P_{\rm surface}$')
ax.set_ylabel(r'$e$')

ax.set_xlim(0.01, 1000)
ax.set_ylim(0, 1)
ax.set_xscale('log')

In [ ]:
# cat = full_catalog
cat = full_catalog[full_catalog['emcee_converged'] & (full_catalog['LOGG'] > -9999)]
# cat = full_catalog[full_catalog['emcee_converged'] & 
#                    np.logical_not(np.isin(full_catalog['APOGEE_ID'], suspicious_ids))]

fig, ax = plt.subplots(1, 1, figsize=(6, 6))

P_surface = period_at_surface(1.35*u.Msun, cat['LOGG'], 
                              cat['e'], 0.4*u.Msun)

ax.scatter(cat['P']/P_surface, cat['LOGG'])
# ax.errorbar((full_catalog['P']/P_surface).value, full_catalog['LOGG'],
#             xerr=(full_catalog['P_err']/P_surface).value, 
#             yerr=full_catalog['LOGG_ERR'],
#             marker='', linestyle='none', zorder=-10)

ax.axvline(1., color='#aaaaaa', zorder=-100)

ax.set_xlabel(r'$P / P_{\rm surface}$')
ax.set_ylabel(r'$\log g$')

ax.set_xlim(0.01, 1000)
ax.set_ylim(4, 0)
ax.set_xscale('log')

In [ ]:
cat = full_catalog[(full_catalog['M2_min_err'] / full_catalog['M2_min']) < 0.2]
# cat = full_catalog[full_catalog['emcee_converged'] & (full_catalog['LOGG'] > -9999)]

fig, ax = plt.subplots(1, 1, figsize=(6, 6))

P_surface = period_at_surface(1.35*u.Msun, cat['LOGG'], 
                              cat['e'], 0.4*u.Msun)

ax.scatter(cat['M1'], cat['M2_min']/cat['M1'])
ax.errorbar(cat['M1'].value, cat['M2_min'].value/cat['M1'].value,
            xerr=cat['M1_err'].value, 
            yerr=cat['M2_min_err'].value,
            marker='', linestyle='none', zorder=-10)

# ax.axvline(1., color='#aaaaaa', zorder=-100)

# ax.set_xlabel(r'$P / P_{\rm surface}$')
# ax.set_ylabel(r'$\log g$')

# ax.set_xlim(0.01, 1000)
ax.set_ylim(0, 1.)
# ax.set_xscale('log')

In [ ]:
session.query(AllStar).join(StarResult, NessRG).filter(StarResult.status_id>0)\
       .filter(StarResult.high_K).count()

In [ ]:
# row = cat[(cat['P']/P_surface) < 3.][1]
# visits = session.query(AllVisit).filter(AllVisit.apogee_id == row['APOGEE_ID']).all()
# visit = visits[0]

In [ ]:
with h5py.File(mcmc_samples_file, 'r') as mcmc_f, h5py.File(samples_file, 'r') as f:
    for row in cat[(cat['P']/P_surface) < 3.]:
        star = AllStar.get_apogee_id(session, row['APOGEE_ID'])
        data = star.apogeervdata()

        if star.apogee_id in mcmc_f:
            samples = JokerSamples.from_hdf5(mcmc_f[star.apogee_id])
            print('mcmc')
        else:
            samples = JokerSamples.from_hdf5(f[star.apogee_id])
            print('thejoker')

        samples.t0 = data.t0

        fig = plot_two_panel(data, samples, 
                             plot_data_orbits_kw=dict(n_times=16384,                
                                                      highlight_P_extrema=False))
        fig.axes[0].set_title('logg={0:.2f}, P = {1:.2f}'.format(star.logg, samples['P'][0]))
        fig.tight_layout()
        
        fig, ax = plt.subplots(1, 1)
        ax.set_title(row['STARFLAGS'], fontsize=12)
        fig = plot_phase_fold(data, MAP_sample(data, samples, joker_params=joker_pars), 
                              ax=ax)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 6))
ax.scatter(m1, mass_ratio, alpha=0.8,
           zorder=10, s=18, color='k')
ax.errorbar(m1.value, mass_ratio, xerr=m1_err.value, yerr=mass_ratio_err, 
            marker='', linestyle='none', alpha=0.75, color='#aaaaaa')
ax.set_xlim(0.5, 2.5)
ax.set_ylim(-0.05, 1.25)

## Now write out the full catalog:

---

# Plots:

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 5))

mask = (full_catalog['emcee_converged'] == True) & (full_catalog['LOGG'] > -999)
tbl2 = full_catalog[mask]

cs = ax.scatter(tbl2['P'], tbl2['e'], c=tbl2['LOGG'], 
                vmin=0.5, vmax=3.5, marker='o', 
                cmap='magma_r', edgecolor='k', 
                linewidth=0.5, s=18)

ax.errorbar(tbl2['P'], tbl2['e'],
            xerr=tbl2['P_err'], yerr=tbl2['e_err'], 
            marker='', linestyle='none', zorder=-10, 
            color='#555555', alpha=0.75)

ax.set_xscale('log')
# ax.set_xlim(2**(0-0.3), 2**(15+0.3))
ax.set_ylim(-0.02, 1.02)

cb = fig.colorbar(cs)
cb.set_label(r'$\log g$')

ax.set_xlabel('period, $P$ [day]')
ax.set_ylabel('eccentricity, $e$')

fig.tight_layout()
fig.set_facecolor('w')
# fig.savefig(path.join(plot_path, 'unimodal-P-e.pdf'))

In [ ]:
from astropy.constants import G
def get_P_surface(M1, logg, e, M2=0*u.Msun):
    P = 2*np.pi * G**(1/4.) * (M1+M2) / (M1**(3/4.)) * (10**logg*u.cm/u.s**2)**(-3/4) * (1-e)**(-3/2)
    return P.to(u.day)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 5))

mask = (full_catalog['emcee_converged'] == True) & (full_catalog['LOGG'] > -999)
tbl2 = full_catalog[mask]

surfaceP = get_P_surface(1.5*u.Msun, tbl2['LOGG'], tbl2['e'])

cs = ax.scatter(tbl2['P']/surfaceP, tbl2['e'], c=tbl2['LOGG'], 
                vmin=0.5, vmax=3.5, marker='o', 
                cmap='magma_r', edgecolor='k', 
                linewidth=0.5, s=18)

ax.errorbar(tbl2['P']/surfaceP, tbl2['e'],
            xerr=tbl2['P_err']/surfaceP, yerr=tbl2['e_err'], 
            marker='', linestyle='none', zorder=-10, 
            color='#555555', alpha=0.75)

ax.scatter((tbl2['P']/surfaceP)[(tbl2['ASPCAPFLAG'] & (2**7)) != 0], 
           tbl2['e'][(tbl2['ASPCAPFLAG'] & (2**7)) != 0], 
           marker='o', edgecolor='tab:red', 
           linewidth=2., s=30, facecolor='none')

ax.set_xscale('log')
# ax.set_xlim(2**(0-0.3), 2**(15+0.3))
ax.set_ylim(-0.02, 1.02)

cb = fig.colorbar(cs)
cb.set_label(r'$\log g$')

# ax.set_xlabel('period, $P$ [day]')
ax.set_ylabel('eccentricity, $e$')

fig.tight_layout()
fig.set_facecolor('w')
# fig.savefig(path.join(plot_path, 'unimodal-P-e.pdf'))

### What's going on at P<10 day?

In [ ]:
from twoface.samples_analysis import max_likelihood_sample

In [ ]:
Plt10 = full_catalog[(full_catalog['P'] < 10.) & (full_catalog['emcee_converged'])]

In [ ]:
with h5py.File(mcmc_samples_file, 'r') as f:
    for apogee_id in Plt10['APOGEE_ID']:
        samples = JokerSamples.from_hdf5(f[apogee_id])
        data = AllStar.get_apogee_id(session, apogee_id).apogeervdata()
        samples.t0 = data.t0
        
        sample = max_likelihood_sample(data, samples)
        print(sample['jitter'])
        
        fig, ax = plt.subplots()
        fig = plot_phase_fold(data, sample)
        ax.set_title(apogee_id)
        fig.tight_layout()